In [ ]:
# We define the data transformations and loaders to load the images and their corresponding
# labels from the training folder. We use the Adam optimizer and cross-entropy loss function for training.
# Finally, we train the model for 10 epochs and save the model weights.
# The training data is loaded using PyTorch's DataLoader, and the model is trained using the Adam optimizer and
# cross-entropy loss. The trained model is then saved to a file for later use.

In [29]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

import matplotlib.pyplot as plt

from model import GenderClassifier

In [22]:
# Define the data transformations and loaders
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
])

train_set = datasets.ImageFolder('mfc_dataset_train_test/train', transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
classes = train_loader.dataset.classes

In [23]:
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")
img = train_features[0].squeeze()
label = train_labels[0]
plt.imshow(img, cmap="gray")
plt.show()
print(f"Label: {label} = {classes[label]}")

In [34]:
# Initialize the model, loss function, and optimizer
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model = GenderClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# Train the model
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

# Save the model
torch.save(model.state_dict(), 'saved_model/weights')


# correct = 0
# total = 0
# with torch.no_grad():
#     for data, labels in test_loader:
#         outputs = model(data)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
#
# print('Accuracy: {} %'.format(100 * correct / total))

ValueError: Expected input batch_size (88) to match target batch_size (32).